In [13]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
# read words

words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [15]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [16]:
# build the dataset

def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context [1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [17]:
len(Xtr), len(Xdev), len(Xte)

(182625, 22655, 22866)

In [18]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27,10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

In [322]:
# learning rate experiment init

# lre = torch.linspace(-3, 0, 1000)
# lrs = 10**lre

In [19]:
for i in range(200000):
    
    #minibatch construct 
    
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    #forward pass
    
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1,30) @ W1 + b1) #32 x 100
    logits = h @ W2 + b2 #32 x 27
    loss = F.cross_entropy(logits, Ytr[ix])
    
    #backward pass

    for p in parameters:
        p.grad = None
    loss.backward()

    #parameter update
    lr = 0.1 if i<100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    #track stats
    ## lri.append(lre[i])
    ## lossi.append(loss.item())

print(loss.item())

2.341378688812256


In [21]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1,30) @ W1 + b1) #32 x 100
logits = h @ W2 + b2 #32 x 27
loss = F.cross_entropy(logits, Ytr)

loss

tensor(2.1972, grad_fn=<NllLossBackward0>)

In [22]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1,30) @ W1 + b1) #32 x 100
logits = h @ W2 + b2 #32 x 27
loss = F.cross_entropy(logits, Ydev)

loss

tensor(2.2520, grad_fn=<NllLossBackward0>)

In [30]:
# sample from the model
block_size = 3
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1,-1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))        
        

carmah.
amorie.
khi.
mili.
taty.
skanna.
eja.
hubena.
jiric.
kaqhiz.
ramara.
chaiiy.
kaleigh.
ham.
join.
quinn.
shoine.
livebi.
waythoniearisi.
jaxenzi.
